In [1]:
# gradio_app.py
import os
import numpy as np
import tensorflow as tf
import gradio as gr
import matplotlib.pyplot as plt

# Load model and test data
model = tf.keras.models.load_model("ann_mnist.h5")
save_dir = os.path.expanduser("~/data")
X_test = np.load(os.path.join(save_dir, 'x_test.npy')).reshape(-1,28,28).astype('float32')/255.0
y_test = np.load(os.path.join(save_dir, 'y_test.npy'))

def predict(img, idx):
    # img: numpy array or None
    if img is None:
        img_arr = X_test[int(idx)]
        true_label = int(y_test[int(idx)])
    else:
        arr = img.astype('float32')
        # if RGB, convert to grayscale
        if arr.ndim == 3:
            arr = arr.mean(axis=2)
        img_arr = tf.image.resize_with_pad(arr[..., None], 28, 28).numpy().squeeze()
        true_label = None
    # Prediction
    probs = model.predict(img_arr.reshape(1,28,28), verbose=0)[0]
    pred = int(np.argmax(probs))
    # Visualization: image + bar chart
    fig, axes = plt.subplots(1, 2, figsize=(6, 3))
    axes[0].imshow(img_arr, cmap='gray'); axes[0].axis('off')
    title = f"Pred→{pred}"
    if true_label is not None:
        title += f" (True:{true_label})"
    axes[0].set_title(title)
    axes[1].bar(range(10), probs)
    axes[1].set_xticks(range(10)); axes[1].set_ylim(0,1)
    axes[1].set_title("Class probabilities")
    plt.tight_layout()
    top3 = {str(i): float(probs[i]) for i in np.argsort(probs)[-3:][::-1]}
    return pred, top3, fig

with gr.Blocks() as demo:
    gr.Markdown("## MNIST Digit Classifier")
    with gr.Row():
        img_input = gr.Image(type='numpy', label='Upload a digit (28×28)')
        idx_slider = gr.Slider(0, len(X_test)-1, value=0, step=1, label='Or pick test sample')
    pred_out = gr.Label(label='Predicted Digit')
    probs_out = gr.Label(num_top_classes=3, label='Top‑3 Probabilities')
    vis_out   = gr.Plot(label='Visualization')

    img_input.change(lambda img, idx: predict(img, idx), [img_input, idx_slider], [pred_out, probs_out, vis_out])
    idx_slider.change(lambda img, idx: predict(img, idx), [img_input, idx_slider], [pred_out, probs_out, vis_out])

if __name__ == '__main__':
    demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report, accuracy_score
import matplotlib.pyplot as plt

#  Load model and test data 
model = tf.keras.models.load_model("tuned_ann_mnist.h5")
save_dir = os.path.expanduser("~/data")
X_test = np.load(os.path.join(save_dir, 'x_test.npy')).reshape(-1, 28, 28).astype('float32') / 255.0
y_test = np.load(os.path.join(save_dir, 'y_test.npy'))

#  Run model predictions 
y_pred_probs = model.predict(X_test, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)

#  Accuracy and Classification Report 
accuracy = accuracy_score(y_test, y_pred)
print(f"\nTest Accuracy: {accuracy:.4f}\n")
print("Classification Report:\n")
print(classification_report(y_test, y_pred, digits=4))

#  Confusion Matrix 
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(10))
disp.plot(cmap='Blues', values_format='d')
plt.title("Confusion Matrix: ANN MNIST Classifier")
plt.show()


313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step

Test Accuracy: 0.9742

Classification Report:

              precision    recall  f1-score   support

           0     0.9837    0.9878    0.9857       980
           1     0.9885    0.9868    0.9877      1135
           2     0.9814    0.9690    0.9751      1032
           3     0.9800    0.9703    0.9751      1010
           4     0.9669    0.9817    0.9742       982
           5     0.9512    0.9843    0.9675       892
           6     0.9820    0.9676    0.9748       958
           7     0.9898    0.9446    0.9667      1028
           8     0.9653    0.9713    0.9683       974
           9     0.9509    0.9792    0.9648      1009

    accuracy                         0.9742     10000
   macro avg     0.9740    0.9742    0.9740     10000
weighted avg     0.9745    0.9742    0.9742     10000



<Figure size 640x480 with 2 Axes>